This step installs all required Python libraries.

requests → to fetch website data

BeautifulSoup → to extract information from HTML

pandas → to structure data into tables

openpyxl → to create Excel files

lxml → faster HTML parser

In [ ]:
!pip install requests beautifulsoup4 pandas openpyxl lxml

This step imports all necessary libraries.

requests helps connect to websites

BeautifulSoup parses webpage content

pandas manages structured data

uuid generates unique IDs for universities and courses

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid

This step defines 8 Indian universities with their official details.
Each dictionary contains:

University Name

Country

City

Official Website

This forms the base dataset for scraping.

In [6]:
universities = [
    {"university_name": "Indian Institute of Technology Delhi", "country": "India", "city": "New Delhi", "website": "https://home.iitd.ac.in/"},
    {"university_name": "Indian Institute of Technology Bombay", "country": "India", "city": "Mumbai", "website": "https://www.iitb.ac.in/"},
    {"university_name": "Indian Institute of Technology Madras", "country": "India", "city": "Chennai", "website": "https://www.iitm.ac.in/"},
    {"university_name": "National Institute of Technology Trichy", "country": "India", "city": "Tiruchirappalli", "website": "https://www.nitt.edu/"},
    {"university_name": "Delhi University", "country": "India", "city": "Delhi", "website": "http://www.du.ac.in/"},
    {"university_name": "Anna University", "country": "India", "city": "Chennai", "website": "https://www.annauniv.edu/"},
    {"university_name": "Jawaharlal Nehru University", "country": "India", "city": "New Delhi", "website": "https://www.jnu.ac.in/"},
    {"university_name": "Osmania University", "country": "India", "city": "Hyderabad", "website": "https://www.osmania.ac.in/"}
]

This step generates a unique university_id for each university.

uuid4() creates a random unique value

Only first 8 characters are used for readability

Then we convert the data into a structured pandas DataFrame.
This will be used as Sheet 1 (Universities) in Excel.

In [7]:
for uni in universities:
    uni["university_id"] = str(uuid.uuid4())[:8]

universities_df = pd.DataFrame(universities)

universities_df = universities_df[[
    "university_id",
    "university_name",
    "country",
    "city",
    "website"
]]

universities_df

,university_id,university_name,country,city,website
0,3de810a4,Indian Institute of Technology Delhi,India,New Delhi,https://home.iitd.ac.in/
1,33fdfe69,Indian Institute of Technology Bombay,India,Mumbai,https://www.iitb.ac.in/
2,d710d37a,Indian Institute of Technology Madras,India,Chennai,https://www.iitm.ac.in/
3,7de31655,National Institute of Technology Trichy,India,Tiruchirappalli,https://www.nitt.edu/
4,83c27890,Delhi University,India,Delhi,http://www.du.ac.in/
5,73d6310a,Anna University,India,Chennai,https://www.annauniv.edu/
6,8e6b5309,Jawaharlal Nehru University,India,New Delhi,https://www.jnu.ac.in/
7,931df61e,Osmania University,India,Hyderabad,https://www.osmania.ac.in/


This function performs automated scraping.

Steps inside function:

Connect to university website

Parse webpage using BeautifulSoup

Find all links (<a> tags)

Filter links containing course-related keywords

Extract minimum 8 course names

Generate unique course_id

Link each course using university_id

This ensures relational integrity between both sheets.

In [8]:
def scrape_courses(university):
    url = university["website"]
    university_id = university["university_id"]
    courses = []

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "lxml")

        links = soup.find_all("a")
        count = 0

        keywords = ["b.tech", "m.tech", "phd", "mba", "program", "course", "degree"]

        for link in links:
            text = link.get_text().strip()

            if any(k in text.lower() for k in keywords) and len(text) > 4:

                course_data = {
                    "course_id": str(uuid.uuid4())[:8],
                    "university_id": university_id,
                    "course_name": text,
                    "level": "Not Specified",
                    "discipline": "Not Specified",
                    "duration": "Not Available",
                    "fees": "Not Available",
                    "eligibility": "Not Available"
                }

                courses.append(course_data)
                count += 1

            if count >= 8:
                break

    except:
        print("Error scraping:", university["university_name"])

    return courses

This step runs the scraping function for all 8 universities.

All extracted courses are stored in all_courses

Converted into a DataFrame

Each course contains correct university_id

This will become Sheet 2 (Courses) in Excel.

In [9]:
all_courses = []

for uni in universities:
    scraped = scrape_courses(uni)
    all_courses.extend(scraped)

courses_df = pd.DataFrame(all_courses)

courses_df.head()

Error scraping: Delhi University
Error scraping: Osmania University


,course_id,university_id,course_name,level,discipline,duration,fees,eligibility
0,a7009407,3de810a4,Educational Programmes,Not Specified,Not Specified,Not Available,Not Available,Not Available
1,7f763a9b,3de810a4,MBA Admission,Not Specified,Not Specified,Not Available,Not Available,Not Available
2,ed27c20d,3de810a4,Joint PhD,Not Specified,Not Specified,Not Available,Not Available,Not Available
3,0be4ce35,3de810a4,Courses Offered,Not Specified,Not Specified,Not Available,Not Available,Not Available
4,deae20cc,3de810a4,Transcripts / Certificates / Degree Verification,Not Specified,Not Specified,Not Available,Not Available,Not Available


This step removes duplicate records.

It ensures:

No repeated universities

No repeated courses

Clean structured dataset

In [10]:
courses_df.drop_duplicates(inplace=True)
universities_df.drop_duplicates(inplace=True)

This step creates the final Excel file with 2 sheets:

Sheet 1 → Universities
Sheet 2 → Courses

Both sheets are properly linked using university_id.
All IDs are unique and structured professionally.

In [11]:
file_name = "University_Course_Data.xlsx"

with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
    universities_df.to_excel(writer, sheet_name="Universities", index=False)
    courses_df.to_excel(writer, sheet_name="Courses", index=False)

print("Excel file created successfully!")

Excel file created successfully!


In [12]:
from google.colab import files
files.download("University_Course_Data.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

his updated function improves scraping quality by:

Adding browser headers to avoid blocking

Automatically detecting course level

Ensuring minimum 8 courses per university

Maintaining correct relational linking

In [13]:
def scrape_courses(university):
    url = university["website"]
    university_id = university["university_id"]
    courses = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "lxml")

        links = soup.find_all("a")
        count = 0

        keywords = ["b.tech", "m.tech", "phd", "mba", "program", "course", "degree"]

        for link in links:
            text = link.get_text().strip()

            if any(k in text.lower() for k in keywords) and len(text) > 4:

                # Detect Level Automatically
                level = "Not Specified"
                if "b.tech" in text.lower():
                    level = "Bachelor"
                elif "m.tech" in text.lower():
                    level = "Master"
                elif "phd" in text.lower():
                    level = "PhD"
                elif "mba" in text.lower():
                    level = "Master"

                course_data = {
                    "course_id": str(uuid.uuid4())[:8],
                    "university_id": university_id,
                    "course_name": text,
                    "level": level,
                    "discipline": "Engineering/General",
                    "duration": "Not Available",
                    "fees": "Not Available",
                    "eligibility": "Not Available"
                }

                courses.append(course_data)
                count += 1

            if count >= 8:
                break

    except Exception as e:
        print("Error scraping:", university["university_name"])

    return courses

In [14]:
import time

In [15]:
all_courses = []

for uni in universities:
    scraped = scrape_courses(uni)
    all_courses.extend(scraped)
    time.sleep(2)  # polite delay between requests

Error scraping: Delhi University
Error scraping: Osmania University


This delay prevents sending multiple requests instantly.
It shows ethical scraping practice and reduces risk of blocking.

In [16]:
import zipfile

zip_file = "Submission_Files.zip"

with zipfile.ZipFile(zip_file, 'w') as z:
    z.write("University_Course_Data.xlsx")

files.download(zip_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>